In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd

In [2]:
# 定义时期
def get_custom_week_period(date):
    start_date = pd.Timestamp('2025-01-02')
    delta_days = (date - start_date).days
    week_number = delta_days // 7
    period_start = start_date + pd.Timedelta(days=week_number * 7)
    period_end = period_start + pd.Timedelta(days=6)
    return f"{period_start.strftime('%Y-%m-%d')}至{period_end.strftime('%Y-%m-%d')}"

In [3]:
# 定义数据处理函数
def process_excel(input_path, output_folder):
    df = pd.read_excel(input_path, sheet_name='日报by渠道')
    df2 = pd.read_excel(input_path, sheet_name='日报by优惠券')    

    df['时期'] = df['日期'].apply(get_custom_week_period)
    weekly_report = df.groupby(['时期','城市']).agg({
        '触达用户数': 'sum',
        '点击用户数': 'sum',
        '领券用户数': 'sum',
        '交易用户数': 'sum',
        '交易订单量': 'sum',
        '核销用户数': 'sum',
        '交易新用户数': 'sum',
        '券补金额': 'sum',
        '渠道成本': 'sum'
    }).reset_index()
    
    weekly_report['未核销用户数'] = weekly_report['交易用户数'] - weekly_report['核销用户数']
    weekly_report['触达点击率'] = weekly_report['点击用户数'] / weekly_report['触达用户数']
    weekly_report['点击转化率'] = weekly_report['交易用户数'] / weekly_report['点击用户数']
    weekly_report['支付核销率'] = weekly_report['核销用户数'] / weekly_report['交易用户数']
    weekly_report['日均交易订单量'] = weekly_report['交易订单量'] / 7
    weekly_report['团单CAC'] = (weekly_report['券补金额'] + weekly_report['渠道成本']) / weekly_report['交易用户数']
    
    rate_columns = ['触达点击率', '点击转化率', '支付核销率']
    weekly_report[rate_columns] = weekly_report[rate_columns].apply(lambda x: (x * 100).round(2).astype(str) + '%')
    
    decimal_columns = ['日均交易订单量', '团单CAC']
    weekly_report[decimal_columns] = weekly_report[decimal_columns].round(2)
    
    cac_report = df.groupby(['时期', '城市','渠道']).agg({
        '渠道成本': 'sum',
        '券补金额': 'sum',
        '交易用户数': 'sum',
        '核销用户数': 'sum'
    }).reset_index()
    
    cac_report['总花费'] = cac_report['渠道成本'] + cac_report['券补金额']
    cac_report['团单CAC'] = cac_report['总花费'] / cac_report['交易用户数']
    cac_report['核销CAC'] = cac_report['总花费'] / cac_report['核销用户数']
    
    decimal_columns2 = ['总花费', '团单CAC', '核销CAC']
    cac_report[decimal_columns2] = cac_report[decimal_columns2].round(2)
    
    conversion_report = df.groupby(['时期','城市', '渠道']).agg({
        '触达用户数': 'sum',
        '点击用户数': 'sum',
        '领券用户数': 'sum',
        '交易用户数': 'sum',
        '交易订单量': 'sum',
        '核销用户数': 'sum',
        '交易新用户数': 'sum',
        '券补金额': 'sum',
        '渠道成本': 'sum'
    }).reset_index()
    
    conversion_report['触达点击率'] = conversion_report['点击用户数'] / conversion_report['触达用户数']
    conversion_report['点击转化率'] = conversion_report['交易用户数'] / conversion_report['点击用户数']
    conversion_report['新用户占比'] = conversion_report['交易新用户数'] / conversion_report['交易用户数']
    
    rate_columns2 = ['触达点击率', '点击转化率', '新用户占比']
    conversion_report[rate_columns2] = conversion_report[rate_columns2].apply(lambda x: (x * 100).round(2).astype(str) + '%')
    
    df2['日期'] = pd.to_datetime(df2['日期'])
    df2['时期'] = df2['日期'].apply(get_custom_week_period)
    df2['是否门店新用户'] = df2['是否新客']
    
    crowd_conversion_report = df2.groupby(['时期', '人群','是否新客','优惠券ID', '面额', '渠道','人群ID','城市']).agg({
        '领券用户数': 'sum',
        '交易用户数': 'sum',
        '交易订单量': 'sum',
        '交易新用户数': 'sum',
        '核销用户数': 'sum',
        '券补金额': 'sum'
    }).reset_index()
    
    channel_user = df2.groupby(['时期','城市', '是否新客', '渠道']).agg({
        '领券用户数': 'sum',
        '交易用户数': 'sum',
        '核销用户数': 'sum',
        '交易订单量': 'sum',
        '交易新用户数': 'sum'
    }).reset_index()
    
    # 定义输出文件路径
    output_file_path = f"{output_folder}/周报.xlsx"
    with pd.ExcelWriter(output_file_path) as writer:
        weekly_report.to_excel(writer, sheet_name='线上获客概览', index=False)
        cac_report.to_excel(writer, sheet_name='CAC', index=False)
        conversion_report.to_excel(writer, sheet_name='线上获客细则', index=False)
        crowd_conversion_report.to_excel(writer, sheet_name='人群漏斗', index=False)
        channel_user.to_excel(writer, sheet_name='渠道分新老', index=False)

    messagebox.showinfo("喵啊", f"完成！文件已保存至 {output_file_path}")

In [4]:
# 打开对话框
def select_file():
    file_path = filedialog.askopenfilename()
    input_entry.delete(0, tk.END)
    input_entry.insert(0, file_path)

def select_folder():
    folder_path = filedialog.askdirectory()
    output_entry.delete(0, tk.END)
    output_entry.insert(0, folder_path)

In [5]:
# GUI配置
root = tk.Tk()
root.title("无忧保洁周报工具")

tk.Label(root, text="选择Excel文件:").grid(row=0, column=0, padx=10, pady=5)
input_entry = tk.Entry(root, width=40)
input_entry.grid(row=0, column=1, padx=10, pady=5)
tk.Button(root, text="浏览", command=select_file).grid(row=0, column=2, padx=10, pady=5)

tk.Label(root, text="选择保存位置:").grid(row=1, column=0, padx=10, pady=5)
output_entry = tk.Entry(root, width=40)
output_entry.grid(row=1, column=1, padx=10, pady=5)
tk.Button(root, text="浏览", command=select_folder).grid(row=1, column=2, padx=10, pady=5)

tk.Button(root, text="运行", command=lambda: process_excel(input_entry.get(), output_entry.get())).grid(row=2, column=1, pady=20)

In [6]:
# 赛马娘启动
root.mainloop()